# API
Web APIS sometimes allows us to request for data, and the data requested is typically returned in the form of XML or JSON.  
## json
JavaScript object notation, JSON, is a lightweight text based syntax for data-interchange.  
More lightweight than XML.
### Malaysian Meteorological Department
https://api.met.gov.my/

In [ ]:
import json

Register for an API key at:  
https://api.met.gov.my/

In [ ]:
key = ''
token = 'METToken '+key

Get the API link after login:  
https://api.met.gov.my/dashboard/docs

In [ ]:
urlstations = 'https://api.met.gov.my/v2/stations'
urllocations = 'https://api.met.gov.my/v2/locations?locationcategoryid=STATE'

Send request, to retrieve json format object

In [ ]:
import requests

In [ ]:
fileobj = requests.get(urllocations, headers={'Authorization':token})
fileobj.text[:300]

Parse JSON object into a dictionary

In [ ]:
dict_text=json.loads(fileobj.text)

Retrieve location id and name:

In [ ]:
locations = {}
for i in dict_text["results"]:
    locations[i["id"]] = i["name"]
locations

Using locations above, now we can retrieve forecast for each state:

In [ ]:
forecasts = pd.Series()
for i, loc in locations.iteritems():
    urlob = 'https://api.met.gov.my/v2/data?datasetid=FORECAST&datacategoryid=GENERAL&locationid='+\
                i+'&start_date=2018-02-19&end_date=2018-02-19'
    fileobj = requests.get(urlob, headers={'Authorization':token})
    dict_text=json.loads(fileobj.text)
    forecasts[loc] = dict_text['results'][0]['value'] 
forecasts

## xml
An alternate response format with APIs is xml format.
## NEA
https://www.nea.gov.sg/api

In [ ]:
import urllib.request as r
from lxml import objectify

Get an API key from NEA:  
http://www.nea.gov.sg/api/api/nea-s-datasets#Register

In [ ]:
key = '781CF461BB6606ADC767F3B357E848ED4A6709A0B39C63D6'

Get API string from nea developer's guide:  
https://www.nea.gov.sg/docs/default-source/api/developer%27s-guide.pdf

In [ ]:
urlNowcast = 'http://api.nea.gov.sg/api/WebAPI/?dataset=2hr_nowcast&keyref=' + key

In [ ]:
# Create request
request = r.Request(urlNowcast)
request.add_header('User-Agent', 'Mozilla/5.0')
response = r.urlopen(request)
response.read()[:500]

Parse response using lxml's objectify:

In [ ]:
parsed = objectify.parse(r.urlopen(request))
root = parsed.getroot()
timestamp = root.item.forecastIssue.get('time')

Loop to retrieve data:

In [ ]:
print('Time ' + timestamp) 
for area in root.iter('area'):
    print(area.get("name") + '  Nowcast:' + area.get('forecast')
         + '  Coordinates:' + area.get("lat") + ',' + area.get("lon"))